In [29]:
import yaml
import pandas as pd
from graphdatascience import GraphDataScience

In [30]:
with open('config.yaml', 'r') as file:
    PARAM = yaml.safe_load(file)


url = "bolt://localhost:7687"
gds = GraphDataScience(url, auth=(PARAM["neo4j_username"], PARAM["neo4j_password"]))

In [31]:
G, result = gds.graph.project(
    "contextGraph",                  #  Graph name
    {
    "Known": { "properties":['protection_status_one_hot', 'site_area_in_hectares', 'site_length_km', 'valuation_methods_one_hot', 'value_year', 'beneficiary_unit_one_hot', 'int__per_hectare_per_year'] },
    "Unknown": {"properties":['protection_status_one_hot', 'site_area_in_hectares', 'site_length_km', 'valuation_methods_one_hot', 'value_year', 'beneficiary_unit_one_hot']},
    "Country": {},
    "Seea": {},
    "Ecosystem": {},
    "Ecozone": {},
    "Biome": {},
    "Cices_class": {},
    "Cices_division": {},
    "Cices_group": {},
    "Cices_section": {},
    "Teeb_service": {},
    "Teeb_subservice": {},
    "Teeb_category": {},
    "Subregion": {},
    "Region": {}
  },                 #  Node projection
  {
    "IS_LOCATE_IN": {"orientation": 'UNDIRECTED'},
    "IS_AN_SEEA_OF": {"orientation": 'UNDIRECTED'},
    "IS_AN_ECOSYSTEM_OF": {"orientation": 'UNDIRECTED'},
    "ECOSYSTEM_BELONGS_TO": {"orientation": 'UNDIRECTED'},
    "IS_AN_ECOZONE_OF": {"orientation": 'UNDIRECTED'},
    "ECOZONE_BELONGS_TO": {"orientation": 'UNDIRECTED'},
    "IS_A_BIOME_OF": {"orientation": 'UNDIRECTED'},
    "BELONGS_TO_CLASS_OF_CICES": {"orientation": 'UNDIRECTED'},
    "IS_A_GROUP_OF_CICES": {"orientation": 'UNDIRECTED'},
    "IS_A_DIVISION_OF_CICES": {"orientation": 'UNDIRECTED'},
    "IS_A_CLASS_OF_CICES": {"orientation": 'UNDIRECTED'},
    "PROVIDES_SERVICE": {"orientation": 'UNDIRECTED'},
    "PROVIDES_SUBSERVICE": {"orientation": 'UNDIRECTED'},
    "COUNTRY_IS_LOCATE_IN": {"orientation": 'UNDIRECTED'},
    "SUBREGION_IS_LOCATE_IN": {"orientation": 'UNDIRECTED'},
    "IS_A_SUBSERVICE_OF": {"orientation": 'UNDIRECTED'},
    "IS_A_SERVICE_OF":  {"orientation": 'UNDIRECTED'}     
    },              #  Relationship projection
    readConcurrency=4           #  Configuration parameters
)

In [32]:
pipe_with_context = gds.nr_pipe("pipe-with-context")

In [33]:
pipe_with_context.addNodeProperty(
    "node2vec",
    mutateProperty="embedding",
    embeddingDimension=32,
    contextNodeLabels=['Country', 'Seea', 'Ecosystem', 'Ecozone', 'Ecosystem', 'Biome','Cices_class', 'Teeb_service', 'Teeb_subservice', 'Subregion', 'Region'],
    iterations=5
)

name                                                 pipe-with-context
nodePropertySteps    [{'name': 'gds.node2vec.mutate', 'config': {'m...
featureProperties                                                   []
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace            {'LinearRegression': [], 'RandomForest': []}
Name: 0, dtype: object

In [34]:
pipe_with_context.selectFeatures(["embedding"])

name                                                 pipe-with-context
nodePropertySteps    [{'name': 'gds.node2vec.mutate', 'config': {'m...
featureProperties                                          [embedding]
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace            {'LinearRegression': [], 'RandomForest': []}
Name: 0, dtype: object

In [35]:
pipe_with_context.addRandomForest(numberOfDecisionTrees=10)

name                                                 pipe-with-context
nodePropertySteps    [{'name': 'gds.node2vec.mutate', 'config': {'m...
featureProperties                                          [embedding]
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace       {'LinearRegression': [], 'RandomForest': [{'ma...
Name: 0, dtype: object

In [36]:
pipe_with_context.configureAutoTuning(maxTrials=5)

name                                                 pipe-with-context
nodePropertySteps    [{'name': 'gds.node2vec.mutate', 'config': {'m...
featureProperties                                          [embedding]
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                      {'maxTrials': 5}
parameterSpace       {'LinearRegression': [], 'RandomForest': [{'ma...
Name: 0, dtype: object

In [37]:
model, stats = pipe_with_context.train(
    G,
    targetNodeLabels=["Known"],
    modelName="nr-pipeline-model-contextual",
    targetProperty="int__per_hectare_per_year",
    metrics=["ROOT_MEAN_SQUARED_ERROR"],
    concurrency=4
)

Node Regression Train Pipeline:   0%|          | 0/100 [00:00<?, ?%/s]

Without the context Node labels: {'test': 40443.38886354173, 'validation': {'min': 43493.441360938836, 'max': 51896.71613829475, 'avg': 48193.77188901938}, 'outerTrain': 39749.880691939696, 'train': {'min': 38593.61942880581, 'max': 44602.9439280394, 'avg': 41371.3436473497}}


In [38]:
print (stats["modelInfo"]["metrics"]["ROOT_MEAN_SQUARED_ERROR"])

{'test': 41704.367138768255, 'validation': {'min': 40784.549552471566, 'max': 54095.044135996526, 'avg': 47100.48149503227}, 'outerTrain': 38406.11450732535, 'train': {'min': 36039.26375185234, 'max': 40014.19000071053, 'avg': 37921.86433286498}}


In [39]:
predicted = model.predict_stream(
    G, modelName="nr-pipeline-model-contextual", targetNodeLabels=["Unknown"]
)
     


Node Classification Predict Pipeline:   0%|          | 0/100 [00:00<?, ?%/s]

In [40]:
predicted["name"] = [x["name"] for x in gds.util.asNodes(predicted["nodeId"].to_list())]

In [41]:
predicted.rename(columns={"predictedValue": "Node2vec_rf"})

,nodeId,Node2vec_rf,name
0,23,68165.58776,12270007
1,31,40722.47882,12250001
2,35,1296.69453,12240002
3,41,94664.04379,12230006
4,46,80655.87647,12210004
...,...,...,...
3471,10291,109097.67564,12410007
3472,10293,4261.83200,12400002
3473,10308,14301.85643,12360001
3474,10315,24295.58599,12350007


In [42]:
predicted.to_csv("predicted_Node2vec.tsv", sep="\t", index=False)